In [3]:
%matplotlib inline
import matplotlib.pyplot as plt
import folium
import requests
import json

import pandas as pd
import numpy as np

## pip install openlostcat
from openlostcat.main_osm_categorizer import MainOsmCategorizer
from ipywidgets import interact

In [4]:
budapest_req = """
[out:json];
    (area["name"="Budapest"];) -> .searchArea;
    nwr[tourism=hotel](area.searchArea);
out tags center;
"""

In [5]:
# pip install openlostcat
from openlostcat.osmqueryutils.ask_osm import ask_osm

budapest_hotels = ask_osm(budapest_req)
len(budapest_hotels['elements'])

260

In [21]:
[nwr.update(nwr['center']) for nwr in budapest_hotels['elements'] if 'center' in nwr]


bp_hotels = pd.DataFrame([[nwr['id'], 
                           nwr['lat'], nwr['lon'], 
                           nwr['tags'].get('name', 'NoName'),  
                           nwr['tags']] for nwr in budapest_hotels['elements']], 
                         columns = ['id', 'lat', 'lng', 'name', 'tags'])

In [26]:
from openlostcat.osmqueryutils.ask_osm import ask_osm_around_point_df

osm = bp_hotels[["lat", "lng"]].apply(lambda x: ask_osm_around_point_df(x, distance = 300), axis = 1)
bp_hotels["osm"] = osm

In [27]:
len(bp_hotels.loc[bp_hotels.osm.isna(), "osm"])

43

In [34]:
if len(bp_hotels.loc[bp_hotels.osm.isna(), "osm"]) > 0:
    osm_isna = bp_hotels[bp_hotels.osm.isna()][["lat", "lng"]].apply(lambda x: ask_osm_around_point_df(x, distance = 300), axis = 1)
    bp_hotels.loc[bp_hotels.osm.isna(), "osm"] = osm_isna

In [35]:
len(bp_hotels.loc[bp_hotels.osm.isna(), "osm"])

0

In [36]:
import folium

zero_location = np.array(list(zip(bp_hotels["lat"],bp_hotels["lng"]))).mean(axis=0)

def show_geo(related_geo, color_map, get_color_func):
    tmp = related_geo
    # extract coordinates
    coords = np.array(list(zip(tmp["lat"],tmp["lng"])))
    # extract other resources
#     Some coding issue: https://github.com/python-visualization/folium/issues/1320
    names = list([str(name.encode('raw_unicode_escape'))[2:-1] for name in tmp.name])
    tags = list([str(str(tag).encode('raw_unicode_escape'))[2:-1] for tag in tmp.tags])
    colors = get_color_func(tmp, color_map)
    m = folium.Map(
        location=coords.mean(axis=0) if len(coords) > 0 else zero_location,
        zoom_start=12,
        tiles='Stamen Terrain'
    )
    for i, loc in enumerate(coords):
        folium.Marker(loc, popup='<i>%s</i>' % tags[i], tooltip=str(names[i]), icon=folium.Icon(color=colors[i])).add_to(m)
    display(m)

## Public transport

In [37]:
categorizer = MainOsmCategorizer('rules/publictransport_rules.json')
print(categorizer.get_categories_enumerated_key_map())
print(categorizer)

bp_hotels["pt_cat"] = [i[0] for i in bp_hotels.osm.map(categorizer.categorize)]

{0: 'pt_primary_accessible', 1: 'pt_accessible', 2: 'pt_nonaccessible'}
CategoryCatalog:
category rule collection: [
    Category name: pt_primary_accessible
    rules: [
        ANY(
            and(
                {public_transport : {'stop_position'}}, is_optional_key = False
                or[
                    {light_rail : {'yes'}}, is_optional_key = False
                    {subway : {'yes'}}, is_optional_key = False
                    {train : {'yes'}}, is_optional_key = False
                ]
            )
        )
    ]
    Category name: pt_accessible
    rules: [
        OR[
            ANY(
                {public_transport : {'stop_position'}}, is_optional_key = False
            )
            ANY(
                {public_transport : {'platform'}}, is_optional_key = False
            )
            ANY(
                {amenity : {'ferry_terminal'}}, is_optional_key = False
            )
        ]
    ]
    Category name: pt_nonaccessible
    rules: [
        CONST

In [38]:
bp_hotels.pt_cat.value_counts()

1    146
0    107
2      7
Name: pt_cat, dtype: int64

In [39]:
color_map_pt = dict(zip([0, 1, 2], ["green", "orange", "red"]))
get_color_func_pt = lambda df, color_map: list(df["pt_cat"].apply(lambda x: color_map.get(x, "black")))
key_map_pt = dict(zip(['pt_primary_accessible', 'pt_accessible', 'pt_nonaccessible'], [0, 1, 2]))



transport = ['All', 'pt_primary_accessible', 'pt_accessible', 'pt_nonaccessible']
@interact(pt_cat=transport)
def get_transport(pt_cat):
    selected = bp_hotels[bp_hotels.pt_cat == key_map_pt[pt_cat]] if pt_cat != 'All' else bp_hotels
    show_geo(selected, color_map_pt, get_color_func_pt)

interactive(children=(Dropdown(description='pt_cat', options=('All', 'pt_primary_accessible', 'pt_accessible',…

## Nearby

In [40]:
categorizer2 = MainOsmCategorizer('rules/nearby.json')
print(categorizer2.get_categories_enumerated_key_map())
print(categorizer2)


nb_cat = [[cat[0] for cat in cat_list] for cat_list in bp_hotels.osm.map(categorizer2.categorize)]
bp_hotels["water_nearby"] = [0 in cats  for cats in nb_cat]
bp_hotels["calm_streets"] = [1 in cats  for cats in nb_cat]

{0: 'water_nearby', 1: 'calm_streets'}
CategoryCatalog:
category rule collection: [
    Category name: water_nearby
    rules: [
        ANY(
            {waterway : {'river'}}, is_optional_key = False
        )
    ]
    Category name: calm_streets
    rules: [
        ALL(
            not(
                {highway : {'primary', 'secondary'}}, is_optional_key = False
            )
        )
    ]
]


In [41]:
print("calm_streets and water: " + str(bp_hotels[bp_hotels.water_nearby & bp_hotels.calm_streets].shape[0]))
print("only water: " + str(bp_hotels[(bp_hotels.water_nearby) & (bp_hotels.calm_streets == False)].shape[0]))
print("only calm_streets: " + str(bp_hotels[(bp_hotels.water_nearby == False) & (bp_hotels.calm_streets)].shape[0]))
print("nothing: " + str(bp_hotels[(bp_hotels.water_nearby == False) & (bp_hotels.calm_streets == False)].shape[0]))

calm_streets and water: 7
only water: 20
only calm_streets: 40
nothing: 193


In [42]:
color_map_nb = {
    (True, False): "blue",
    (False, True): "green",
    (True, True): "purple",
    (False, False): "black"
}
get_color_func_nb = lambda df, color_map: list(map(lambda x: color_map.get(x, "black"), list(zip(df.water_nearby, df.calm_streets))))
nearby = ['All', 'water_nearby', 'calm_streets', 'both', 'none']
@interact(nearby_cat=nearby)
def get_nerby(nearby_cat):
    water_nearby = lambda x: x[x.water_nearby]
    calm_streets = lambda x: x[x.calm_streets]
    both = lambda x: x[x.water_nearby & x.calm_streets]
    none = lambda x: x[(x.water_nearby == False) & (x.calm_streets == False)]
    switch = {
        'water_nearby': water_nearby,
        'calm_streets': calm_streets,
        'both': both,
        'none': none
    }
    selected = switch.get(nearby_cat, lambda x: x)(bp_hotels)
    show_geo(selected, color_map_nb, get_color_func_nb)

interactive(children=(Dropdown(description='nearby_cat', options=('All', 'water_nearby', 'calm_streets', 'both…

## Preferred: Mix

In [43]:
categorizer3 = MainOsmCategorizer('rules/mix.json')
print(categorizer3.get_categories_enumerated_key_map())
print(categorizer3)

bp_hotels["preferred_cat"] = [i[0] for i in bp_hotels.osm.map(categorizer3.categorize)]

{0: '1st_preferred_location', 1: '2nd_preferred_location', 2: '3rd_preferred_location', 3: 'not_preferred_location'}
CategoryCatalog:
category rule collection: [
    Category name: 1st_preferred_location
    rules: [
        AND(
            REF ##pt_primary_accessible(
                ANY(
                    and(
                        {public_transport : {'stop_position'}}, is_optional_key = False
                        or[
                            {light_rail : {'yes'}}, is_optional_key = False
                            {subway : {'yes'}}, is_optional_key = False
                            {train : {'yes'}}, is_optional_key = False
                        ]
                    )
                )
            )
            REF ##water_OR_calm_streets(
                OR[
                    REF ##water_nearby(
                        ANY(
                            {waterway : {'river'}}, is_optional_key = False
                        )
                    )
              

In [44]:
bp_hotels.preferred_cat.value_counts()

3    193
2     49
0     12
1      6
Name: preferred_cat, dtype: int64

In [45]:
color_map_mix = dict(zip([0, 1, 2, 3], ["green", "blue", "orange", "black"]))
get_color_func_mix = lambda df, color_map: list(df["preferred_cat"].apply(lambda x: color_map.get(x, "black")))
key_map_mix = dict(zip(['1st_preferred_location', '2nd_preferred_location', '3rd_preferred_location', 'not_preferred_location'], [0, 1, 2, 3]))

preffered = ['All', '1st_preferred_location', '2nd_preferred_location', '3rd_preferred_location', 'not_preferred_location']
@interact(preffered_cat=preffered)
def get_nerby(preffered_cat):
    selected = bp_hotels[bp_hotels.preferred_cat == key_map_mix[preffered_cat]] if preffered_cat != 'All' else bp_hotels
    show_geo(selected, color_map_mix, get_color_func_mix)

interactive(children=(Dropdown(description='preffered_cat', options=('All', '1st_preferred_location', '2nd_pre…